In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
df_train.head()

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
df_test.head()

In [ ]:
df_testing = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
#df_testing.head()

In [ ]:
print(df_train.isnull().sum())
print(df_test.isnull().sum())

In [ ]:
df_train.describe()

In [ ]:
df_train.drop(['id'],axis=1,inplace=True)
df_test.drop(['id'],axis=1,inplace=True)

In [ ]:
sns.set_style("dark")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(9, 8))

sns.distplot(df_train['target'], color="b")
ax.xaxis.grid(False)
ax.set(ylabel="Values")
ax.set(xlabel="Target")
ax.set(title="Target distribution")
sns.despine(trim=True, left=True)
plt.show()

In [ ]:
features = [f'cont{x}'for x in range(1,15)]

In [ ]:
# we will look into the features distribution now, to get insight into the data, 
#that how train and test feature corelated with each other!
i = 1
plt.figure()
fig, ax = plt.subplots(5, 3,figsize=(14, 24))
for feature in features:
    plt.subplot(5, 3,i)
    sns.distplot(df_train[feature],color="blue", kde=True,bins=120, label='train')
    sns.distplot(df_test[feature],color="green", kde=True,bins=120, label='test')
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
df_train.corr()

We'll be plotting the heatmap to see the co-relations!

In [ ]:
corr = df_train.corr()
plt.subplots(figsize=(14,10))
sns.heatmap(corr, vmax=0.9, cmap="viridis", square=True)

In [ ]:
# Plot boxplots for all features to find outliers

for col in df_train.columns[:-1]:
    plt.boxplot([df_train[col], df_test[col]], labels=['train', 'test'])
    plt.title(col)
    plt.legend()
    plt.show()

In [ ]:
plt.boxplot(df_train['target'])

In [ ]:
Q1_train = df_train.quantile(0.25)
Q3_train = df_train.quantile(0.75)
IQR_train = Q3_train - Q1_train
((df_train < Q1_train - 1.5*IQR_train) | (df_train > Q3_train + 1.5*IQR_train)).agg([sum, 'mean'])

In [ ]:
Q1_test = df_test.quantile(0.25)
Q3_test = df_test.quantile(0.75)
IQR_test = Q3_test - Q1_test
((df_test < Q1_test - 1.5 * IQR_test) | (df_test > Q3_test + 1.5 * IQR_test)).agg([sum, 'mean'])

After setting up the quantile ranges, we can conclude that features cont7, cont9 and cont10(In train) also the target label are having the significant outliers present!
We'll try remove these outliers present!

In [ ]:
def replace_outliers(data):
    for col in data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        median_ = data[col].median()
#         data[col].mask(((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)), median_, inplace=True)
        # data[col] = np.where(((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)),
        #                     median_, data[col])
        data.loc[((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)), col] = median_
    return data

In [ ]:
df_train = replace_outliers(df_train)

In [ ]:
sns.set_style("dark")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(9, 8))

sns.distplot(df_train['target'], color="b")
ax.xaxis.grid(False)
ax.set(ylabel="Values")
ax.set(xlabel="Target")
ax.set(title="Target distribution")
sns.despine(trim=True, left=True)
plt.show()

As, we can see the target distribution before and after removing the outlier!

In [ ]:
# df_train['contS'] = df_train['cont2'] * df_train['cont9']
# df_test['contS'] = df_test['cont2'] * df_test['cont9']

# df_train['contR'] = df_train['cont3'] * df_train['cont11']
# df_test['contR'] = df_test['cont3'] * df_test['cont11']

# df_train['contG'] = df_train['cont1'] * df_train['cont7']
# df_test['contG'] = df_test['cont1'] * df_test['cont7']

In [ ]:
# df_train.drop(['cont11'],axis=1,inplace=True)
# df_test.drop(['cont11'],axis=1,inplace=True)

# df_train.drop(['cont9'],axis=1,inplace=True)
# df_test.drop(['cont9'],axis=1,inplace=True)

# df_train.drop(['cont7'],axis=1,inplace=True)
# df_test.drop(['cont7'],axis=1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
X=df_train.drop(['target'],axis=1)
y=df_train.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.18, random_state=33)

In [ ]:
# import lightgbm as lgb
# from sklearn.model_selection import GridSearchCV
# parameters = {'depth'         : [4,6,8,10,12,14,18,20],
#                   'learning_rate' : [0.005, 0.01, 0.035, 0.05, 0.1, 0.15, 0.2],
#                   'iterations'    : [300, 800, 1000, 1800, 3000, 4100, 5000]
#                  }
# LGB = lgb.LGBMRegressor()

# grid = GridSearchCV(estimator=LGB, param_grid = parameters, cv = 3, n_jobs=-1)
# grid.fit(X, y)
# CBR
# print(" Results from Grid Search " )
# print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
# print("\n The best score across ALL searched params:\n", grid.best_score_)
# print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
# LGB_Test = lgb.LGBMRegressor(depth=10, iterations=100, learning_rate=0.1)

# LGB_Test.fit(X,y)

Derieved these hyperparameters after referring some other notebooks as well as executing the hyperparameter tuning, and selecting the best parameters for our model!

In [ ]:
import lightgbm as lgb
LGB = lgb.LGBMRegressor(random_state=33, n_estimators=5000, min_data_per_group=5, boosting_type='gbdt',
 num_leaves=246, max_dept=-1, learning_rate=0.005, subsample_for_bin=200000,
 lambda_l1= 1.074622455507616e-05, lambda_l2= 2.0521330798729704e-06, n_jobs=-1, cat_smooth=1.0, 
 importance_type='split', metric='rmse', min_child_samples=20, min_gain_to_split=0.0, feature_fraction=0.5, 
 bagging_freq=6, min_sum_hessian_in_leaf=0.001, min_data_in_leaf=100, bagging_fraction=0.82063411)

LGB.fit(X_train, y_train)

In [ ]:
pred_LGB = LGB.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse_LGB = np.sqrt(mean_squared_error(y_test, pred_LGB))
rmse_LGB

In [ ]:
pred_LGB = LGB.predict(df_test)
pred_LGB

In [ ]:
# output = pd.DataFrame({'id': df_testing.id, 'target': pred_LGB})
# output.to_csv('Kaggle_Playground_Submission.csv', index=False)

### Do Up-Vote! If this was helpful!